### Analysing masters present in bug reports

In [1]:
!pip install networkx

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import _pickle as pickle

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

In [5]:
# Domain to use
DOMAIN = 'openoffice'
'''
    propose_centroid_bert_
    propose_bert_triplet_
    propose_bert_
    baseline_dwen_
    baseline_
'''
METHOD = 'DMS_QL_{}'.format(epochs)
EMBED_METHOD='bert'
ONLY_BUCKETS = False
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [6]:
path_buckets = 'data/normalized/{}/{}.csv'.format(DOMAIN, DOMAIN)

df = pd.read_csv(path_buckets)

In [7]:
df.head()

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
0,14,trivial,CLOSED,Website general issues,2000-10-17 19:40:00 +0000,2006-02-07 22:23:55 +0000,it would be nice if the combination of OpenOff...,[],P3,Infrastructure,FIXED,openoffice.org issuezillla URL's display a hor...,current
1,15,trivial,CLOSED,Bugzilla,2000-10-19 11:49:00 +0000,2003-12-27 10:23:17 +0000,this task is just a test\nthis task is just at...,[],P5,Infrastructure,FIXED,test task,current
2,16,trivial,CLOSED,_openoffice.org administrative interface (obso...,2000-10-20 15:06:00 +0000,2003-12-06 14:52:32 +0000,Using the ADMIN link on the main page leads to...,[],P1,Infrastructure,FIXED,Administrative Interface not available,current
3,17,trivial,CLOSED,_openoffice.org CVS (obsolete),2000-10-20 18:46:00 +0000,2003-12-06 14:52:32 +0000,Entered into Issuezilla for tracking purposes....,[],P3,Infrastructure,FIXED,update modules file.,current
4,19,trivial,CLOSED,definition,2000-10-20 22:54:00 +0000,2007-09-22 22:15:55 +0000,<text:quo-vadis></text:quo-vadis>\n <text:er...,[],P2,xml,FIXED,"text:quo-vadis, text:ergo-sum",605


In [8]:
G=nx.Graph()
for row in df.iterrows():
    bug_id = row[1]['bug_id']
    dup_id = row[1]['dup_id']
    if dup_id == '[]':
        G.add_node(dup_id)
    else:
        G.add_edges_from([(int(bug_id), int(dup_id))])

In [9]:
for r in nx.connected_components(G):
    print(r)

{49152, 49146}
{90791, 98306, 50853, 33630, 88871}
{32771, 32490, 33548, 32560, 33879, 32699}
{32772, 22694, 36970, 39820, 33298, 36760, 34488, 34911}
{15712, 13193}
{98206, 98310}
{32776, 30241, 33762, 31134, 33183}
{65545, 55967}
{27332, 32780, 37094}
{26122, 65549, 3411, 94365, 74204, 62365}
{30434, 32782}
{107242, 102941, 98319}
{25223, 26888, 62344, 29706, 101130, 19341, 20878, 9485, 30608, 94736, 50964, 2197, 88980, 19226, 51742, 70047, 20512, 76324, 99750, 69927, 33077, 65347, 101956, 7624, 24784, 89297, 61269, 3927, 50020, 24293, 25841, 68086, 8823, 88824, 88191}
{87384, 86209}
{18, 100, 271}
{31730, 32787}
{31681, 32788, 32789, 31726}
{21280, 50753, 35298, 27520, 85187, 51168, 83046, 27630, 81041, 46738, 46739, 46740, 103094, 48951, 65561, 55995, 76895}
{69481, 65562, 65597}
{59, 27, 92}
{53049, 54618}
{56174, 58491, 18620, 65566, 61775}
{87786, 85109}
{32, 42944, 18886, 3207, 6342, 32905, 54854, 54855, 67697, 4756, 23861, 18648, 58138, 40286}
{101545, 103282, 103771}
{100385,

{12198, 12199}
{8661, 7663}
{47907, 55891}
{40443, 41358}
{49730, 40444}
{96162, 99431, 94124, 95085, 95630, 96724, 96445}
{45233, 45234, 40459, 40628}
{116656, 116056, 115837}
{8892, 7695}
{38144, 40467}
{69069, 91269}
{40480, 40762}
{40481, 41713}
{40700, 40487}
{73261, 73262}
{40496, 40498}
{41446, 40503}
{72272, 73805}
{40868, 40550, 41167, 41209, 40666, 40508}
{40510, 44007}
{106168, 106052}
{106596, 106057, 106410, 106142, 106607, 106512, 106159, 106779, 106165, 106166, 106331, 106622, 106495}
{46329, 44340, 40533, 43004}
{7768, 9393}
{73312, 74789}
{7777, 7779}
{48484, 75075, 48132, 106085, 98095}
{16411, 10323, 7796}
{24097, 9762, 24821, 7798}
{7800, 8039}
{73337, 73338}
{106108, 106804}
{42740, 40582}
{9446, 7815}
{20368, 7817, 17996, 20249, 16444}
{50458, 50455}
{90824, 90925, 89487, 92721, 89243, 90843, 91103}
{101333, 102181}
{7825, 8028}
{40600, 40619}
{41880, 40602}
{73595, 73372}
{53568, 40610, 31948}
{7874, 7846, 9702, 9648, 9685, 9655, 9658}
{105760, 105640, 106158}
{1

{120563, 120564}
{61913, 22268}
{102243, 102059, 101919}
{108290, 107477}
{89067, 87820}
{69250, 65463}
{22306, 22373}
{22336, 27689, 23769}
{27432, 16011, 55132, 55134, 30631}
{86516, 78055}
{86071, 83671}
{120696, 119711}
{55164, 55165}
{23011, 22399}
{56752, 55169}
{28232, 22402, 25443, 25444}
{87047, 87943}
{87944, 90100}
{87954, 88477, 87966}
{22689, 22422}
{22424, 22423}
{56403, 55204}
{91550, 92331, 87974}
{23272, 22447}
{92016, 98949}
{24584, 22450}
{22452, 22613, 24502, 24631}
{22524, 22462}
{91160, 91125}
{55232, 55322, 55383, 55335}
{22466, 22467}
{87003, 86676}
{55272, 55273, 55355}
{22512, 22513}
{22825, 65006, 57393, 56563, 56564, 56565, 84534, 22519}
{56381, 55294, 60223}
{94900, 96597}
{22552, 27307, 28639}
{24753, 22558}
{90473, 96045}
{67901, 74758}
{89371, 88117}
{88120, 85365}
{84921, 85684}
{63844, 69334}
{47802, 55386}
{88160, 89217}
{31409, 22638}
{89926, 91303, 88176, 90582, 90077, 90463}
{55429, 55966}
{89136, 89352}
{64369, 63852}
{22667, 23684}
{25820, 22670}

In [10]:
nx.degree(G)

DegreeView({16384: 1, 98306: 1, 32771: 3, 32772: 7, 15712: 1, 98310: 1, 32776: 1, 65545: 1, 32780: 2, 65549: 3, 32782: 1, 98319: 2, 54616: 3, 18: 1, 32787: 1, 32788: 1, 32789: 1, 65561: 1, 65562: 1, 27: 2, 21850: 1, 65566: 1, 32: 1, 58922: 1, 65578: 1, 43: 1, 44: 1, 45: 1, 32814: 3, 32815: 1, 65586: 1, 51: 1, 52: 1, 53: 1, 54: 1, 32825: 1, 32826: 1, 59: 1, 32829: 1, 51285: 1, 63: 1, 64: 1, 98369: 1, 32834: 1, 32835: 1, 65604: 1, 65605: 1, 65608: 1, 32842: 1, 75: 1, 32781: 2, 65616: 1, 32850: 3, 84: 1, 98318: 1, 98391: 3, 32857: 1, 98394: 1, 31643: 2, 92: 1, 32861: 1, 8732: 1, 32863: 1, 100: 2, 32870: 1, 32872: 1, 105: 1, 65643: 1, 29984: 1, 32879: 1, 32880: 1, 32883: 1, 121: 1, 65658: 1, 17462: 1, 32892: 1, 21866: 1, 65557: 1, 32896: 1, 32898: 1, 32899: 1, 98436: 1, 22918: 1, 32903: 1, 32905: 1, 98442: 1, 65676: 1, 32910: 1, 98447: 1, 98448: 1, 98451: 1, 98452: 1, 32917: 1, 32920: 1, 32921: 3, 65690: 8, 50197: 2, 159: 3, 32928: 1, 65697: 2, 32930: 1, 32931: 1, 27885: 1, 32933: 1, 32935

In [11]:
#nx.draw(G)

In [12]:
stats = {}
for r in nx.connected_components(G):
    group_size = len(r)
    if group_size in stats:
        stats[group_size] += 1
    else:
        stats[group_size] = 1

In [13]:
pd.DataFrame(stats, index=[1]).transpose().rename(columns={1 : 'size_group'})\
    .sort_values(by='size_group', ascending=False).reset_index().rename(columns={'index' : 'group'})

,group,size_group
0,2,4412
1,3,1109
2,4,436
3,5,265
4,6,153
5,7,101
6,8,69
7,9,49
8,11,29
9,10,25


In [14]:
def create_bucket(df):
        print("Creating the buckets...")
        buckets = {}
        G=nx.Graph()
        for row in tqdm(df.iterrows()):
            bug_id = row[1]['bug_id']
            dup_id = row[1]['dup_id']
            if dup_id == '[]':
                G.add_node(bug_id)
            else:
                G.add_edges_from([(int(bug_id), int(dup_id))])
        for g in tqdm(nx.connected_components(G)):
            group = set(g)
            for bug in g:
                master = int(bug)
                query = df[df['bug_id'] == master]
                if query.shape[0] <= 0:
                    group.remove(master)
                    master = np.random.choice(list(group), 1)
                buckets[int(master)] = group
        return buckets

In [15]:
buckets = create_bucket(df)

Creating the buckets...


In [16]:
len(buckets)

72234

In [17]:
with open(os.path.join(DIR, DOMAIN + '_buckets.pkl'), 'wb') as f:
    pickle.dump(buckets, f)
print("Buckets saved")

Buckets saved
